# Label Propagation - Event Classification
This notebook establishes a training pipeline for our Event Embedding model.

In [ ]:
import networkx as nx
from tqdm import tqdm
import pandas as pd


# auth.authenticate_user()
# print('Authenticated')

## 5W1H Graph Events

In [ ]:
!gdown --id 1RF_bIo5ndxPhu9SJw-T8HBcuHyaGQGL0

Downloading...
From: https://drive.google.com/uc?id=1RF_bIo5ndxPhu9SJw-T8HBcuHyaGQGL0
To: /content/datasets.tar.gz
22.7MB [00:00, 85.7MB/s]


In [ ]:
!tar -xzvf datasets.tar.gz

datasets_runs/
datasets_runs/run_1_google_news_5w1h_graph_hin.nx
datasets_runs/run_6_40er_5w1h_graph_hin.nx
datasets_runs/run_4_bbc_5w1h_graph_hin.nx
datasets_runs/run_8_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_5_bbc_5w1h_graph_hin.nx
datasets_runs/run_9_google_news_5w1h_graph_hin.nx
datasets_runs/run_5_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_2_bbc_5w1h_graph_hin.nx
datasets_runs/run_9_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_7_40er_5w1h_graph_hin.nx
datasets_runs/run_9_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_8_google_news_5w1h_graph_hin.nx
datasets_runs/run_10_bbc_5w1h_graph_hin.nx
datasets_runs/run_8_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_2_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_8_40er_5w1h_graph_hin.nx
datasets_runs/run_6_bbc_5w1h_graph_hin.nx
datasets_runs/run_4_google_news_5w1h_graph_hin.nx
datasets_runs/run_2_google_news_5w1h_graph_hin.nx
datasets_runs/run_7_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_4_gold_standard_

# Regularization

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import random


def LP(G,iterations=30):

    label_codes = {}
    for node in G.nodes():
      if 'train' in G.nodes[node]:
        label = G.nodes[node]['label']
        if label not in label_codes: label_codes[label] = len(label_codes)

    num_labels = len(label_codes)
    for node in G.nodes():
      if 'train' in G.nodes[node]:
        G.nodes[node]['y'] = np.array([0.0]*num_labels)
        label = G.nodes[node]['label']
        G.nodes[node]['y'][label_codes[label]] = 1.0
    
    nodes = []

    # inicializando vetor f para todos os nodes
    for node in G.nodes():
        G.nodes[node]['f'] = np.array([0.0]*num_labels)
        if 'y' in G.nodes[node]:
            G.nodes[node]['f'] = G.nodes[node]['y']*1.0
        nodes.append(node)

    pbar = tqdm(range(0, iterations))

    for iteration in pbar:
        random.shuffle(nodes)
        energy = 0.0

        # percorrendo cada node
        for node in nodes:
            f_new = np.array([0.0]*num_labels)
            f_old = np.array(G.nodes[node]['f'])*1.0
            sum_w = 0.0

            # percorrendo vizinhos do onde
            for neighbor in G.neighbors(node):
                w = 1.0
                if 'weight' in G[node][neighbor]:
                    w = G[node][neighbor]['weight']

                w /= np.sqrt(G.degree[neighbor])

                f_new += w*G.nodes[neighbor]['f']

                sum_w += w

            f_new /= sum_w

            G.nodes[node]['f'] = f_new*1.0

            if 'y' in G.nodes[node]:
                G.nodes[node]['f'] = G.nodes[node]['y']*1.0

            energy += np.linalg.norm(f_new-f_old)

        iteration += 1
        message = 'Iteration '+str(iteration)+' | Energy = '+str(energy)
        pbar.set_description(message)

    return G, label_codes

In [ ]:
G = nx.read_gpickle('datasets_runs/run_3_40er_5w1h_graph_hin.nx')
G

In [ ]:
G, label_codes = LP(G)

In [ ]:
label_codes

{'business': 0, 'technology': 2, 'world': 1}

In [ ]:
y_true = []
y_pred = []
for node in G.nodes():
  if 'test' in G.nodes[node]:
    y_true.append(G.nodes[node]['label'])
    cod_predicted = np.argmax(G.nodes[node]['f'])
    label_predicted = None
    for label in label_codes:
      if label_codes[label]==cod_predicted:
        label_predicted = label
        break
    y_pred.append(label_predicted)

In [ ]:
print(y_true)
print(y_pred)

['business', 'world', 'world', 'business', 'technology', 'world', 'business', 'world', 'business', 'world', 'world', 'world', 'world', 'technology', 'world', 'business', 'world', 'technology', 'technology', 'business', 'business', 'business', 'technology', 'business', 'world', 'world', 'world', 'business', 'world', 'world', 'world', 'business']
['business', 'world', 'world', 'business', 'world', 'world', 'business', 'world', 'business', 'world', 'world', 'world', 'world', 'technology', 'world', 'business', 'world', 'world', 'world', 'business', 'business', 'business', 'world', 'business', 'world', 'world', 'world', 'business', 'world', 'world', 'world', 'business']


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

f1_macro = f1_score(y_true, y_pred, average='macro')
acc = accuracy_score(y_true, y_pred)

network_file = ''
print('--->' ,network_file,'f1_macro',f1_macro,'acc',acc)
#experimental_results.append((network_file,'f1_macro',f1_macro,'acc',acc,y_true,y_pred))

--->  f1_macro 0.7407407407407408 acc 0.875


In [ ]:
from os import listdir
from os.path import isfile, join
path_datasets = 'datasets_runs/'
network_files = [f for f in listdir(path_datasets) if isfile(join(path_datasets, f))]
print(network_files)

['run_1_google_news_5w1h_graph_hin.nx', 'run_6_40er_5w1h_graph_hin.nx', 'run_4_bbc_5w1h_graph_hin.nx', 'run_8_gold_standard_5w1h_graph_hin.nx', 'run_5_bbc_5w1h_graph_hin.nx', 'run_9_google_news_5w1h_graph_hin.nx', 'run_5_gold_standard_5w1h_graph_hin.nx', 'run_2_bbc_5w1h_graph_hin.nx', 'run_9_news_cluster_5w1h_graph_hin.nx', 'run_7_40er_5w1h_graph_hin.nx', 'run_9_gold_standard_5w1h_graph_hin.nx', 'run_8_google_news_5w1h_graph_hin.nx', 'run_10_bbc_5w1h_graph_hin.nx', 'run_8_news_cluster_5w1h_graph_hin.nx', 'run_2_news_cluster_5w1h_graph_hin.nx', 'run_8_40er_5w1h_graph_hin.nx', 'run_6_bbc_5w1h_graph_hin.nx', 'run_4_google_news_5w1h_graph_hin.nx', 'run_2_google_news_5w1h_graph_hin.nx', 'run_7_gold_standard_5w1h_graph_hin.nx', 'run_4_gold_standard_5w1h_graph_hin.nx', 'run_5_40er_5w1h_graph_hin.nx', 'run_3_gold_standard_5w1h_graph_hin.nx', 'run_4_40er_5w1h_graph_hin.nx', 'run_5_google_news_5w1h_graph_hin.nx', 'run_10_news_cluster_5w1h_graph_hin.nx', 'run_10_40er_5w1h_graph_hin.nx', 'run_9_40

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm

experimental_results = []

for network_file in tqdm(network_files):

  print('Networkfile',network_file)
  
  G = nx.read_gpickle(path_datasets+network_file)

  G, label_codes = LP(G)

  y_true = []
  y_pred = []
  for node in G.nodes():
    if 'test' in G.nodes[node]:
      y_true.append(G.nodes[node]['label'])
      cod_predicted = np.argmax(G.nodes[node]['f'])
      label_predicted = None
      for label in label_codes:
        if label_codes[label]==cod_predicted:
          label_predicted = label
          break
      y_pred.append(label_predicted)

  f1_macro = f1_score(y_true, y_pred, average='macro')
  acc = accuracy_score(y_true, y_pred)

  print('--->' ,network_file,'f1_macro',f1_macro,'acc',acc)
  experimental_results.append((network_file,'f1_macro',f1_macro,'acc',acc,y_true,y_pred))

Networkfile run_1_google_news_5w1h_graph_hin.nx



---> run_1_google_news_5w1h_graph_hin.nx f1_macro 0.7985375968569246 acc 0.84
Networkfile run_6_40er_5w1h_graph_hin.nx



---> run_6_40er_5w1h_graph_hin.nx f1_macro 0.7508547008547009 acc 0.75
Networkfile run_4_bbc_5w1h_graph_hin.nx



---> run_4_bbc_5w1h_graph_hin.nx f1_macro 0.4757027286439051 acc 0.4772727272727273
Networkfile run_8_gold_standard_5w1h_graph_hin.nx



---> run_8_gold_standard_5w1h_graph_hin.nx f1_macro 0.5323863636363636 acc 0.6973684210526315
Networkfile run_5_bbc_5w1h_graph_hin.nx


...


---> run_8_bbc_5w1h_graph_hin.nx f1_macro 0.44988344988344997 acc 0.45454545454545453
Networkfile run_6_news_cluster_5w1h_graph_hin.nx



---> run_6_news_cluster_5w1h_graph_hin.nx f1_macro 0.5497998693479325 acc 0.6535947712418301
Networkfile run_1_40er_5w1h_graph_hin.nx



---> run_1_40er_5w1h_graph_hin.nx f1_macro 0.47509578544061304 acc 0.65625
Networkfile run_7_news_cluster_5w1h_graph_hin.nx



---> run_7_news_cluster_5w1h_graph_hin.nx f1_macro 0.5084017981876038 acc 0.6241830065359477
Networkfile run_3_40er_5w1h_graph_hin.nx



---> run_3_40er_5w1h_graph_hin.nx f1_macro 0.7407407407407408 acc 0.875



In [ ]:
df_results = pd.DataFrame(experimental_results)
df_results

,0,1,2,3,4,5,6
0,run_1_google_news_5w1h_graph_hin.nx,f1_macro,0.798538,acc,0.840000,"[Equifax breach, IPhone-X Green-Line, Star War...","[North Korea Missile, IPhone-X Green-Line, Sta..."
1,run_6_40er_5w1h_graph_hin.nx,f1_macro,0.750855,acc,0.750000,"[business, world, world, business, business, t...","[business, world, world, business, business, b..."
2,run_4_bbc_5w1h_graph_hin.nx,f1_macro,0.475703,acc,0.477273,"[business, politics, sport, business, politics...","[business, politics, sport, entertainment, tec..."
3,run_8_gold_standard_5w1h_graph_hin.nx,f1_macro,0.532386,acc,0.697368,"[toberone-gate, clinton blames comey, cubs win...","[toberone-gate, clinton blames comey, clinton ..."
4,run_5_bbc_5w1h_graph_hin.nx,f1_macro,0.462235,acc,0.454545,"[business, politics, sport, business, politics...","[business, tech, sport, tech, tech, tech, poli..."
5,run_9_google_news_5w1h_graph_hin.nx,f1_macro,0.671769,acc,0.720000,"[IPhone-X Green-Line, Dieselgate, IPhone-X Gre...","[IPhone-X Green-Line, Star Wars: Battlefront I..."
6,run_5_gold_standard_5w1h_graph_hin.nx,f1_macro,0.573206,acc,0.684211,"[toberone-gate, clinton blames comey, china we...","[toberone-gate, clinton blames comey, toberone..."
7,run_2_bbc_5w1h_graph_hin.nx,f1_macro,0.317308,acc,0.363636,"[business, politics, politics, sport, business...","[entertainment, tech, politics, politics, tech..."
8,run_9_news_cluster_5w1h_graph_hin.nx,f1_macro,0.495744,acc,0.620915,"[harambe, The_New_Infinity_War_Trailer, elon_m...","[harambe, thousands_flee_fireworks_explosion_a..."
9,run_7_40er_5w1h_graph_hin.nx,f1_macro,0.544678,acc,0.750000,"[business, business, world, business, business...","[business, business, world, business, business..."


In [ ]:
df_results.to_excel('LP_results_r1.xls')